# MLND Capstone project

## Fire up graphlab create & Import dataset


In [1]:
import graphlab as gl
from datetime import datetime
app_labels = gl.SFrame.read_csv('source_data/app_labels.csv',column_type_hints={'app_id': str})
gender_age_train = gl.SFrame.read_csv('source_data/gender_age_train.csv',column_type_hints={'device_id': str})
label_categories = gl.SFrame.read_csv('source_data/label_categories.csv')
phone_brand_device_model = gl.SFrame.read_csv('source_data/phone_brand_device_model.csv',column_type_hints={'device_id': str})
sample_submission = gl.SFrame.read_csv('source_data/sample_submission.csv',column_type_hints={'device_id': str})
app_events = gl.SFrame.read_csv('source_data/app_events.csv',column_type_hints={'app_id': str})
events = gl.SFrame.read_csv('source_data/events.csv',column_type_hints={'device_id': str})

A newer version of GraphLab Create (v2.1) is available! Your current version is v1.9.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


This non-commercial license of GraphLab Create for academic use is assigned to rufang@ucsd.edu and will expire on May 08, 2017.


2016-07-31 20:21:43,786 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.9 started. Logging: /tmp/graphlab_server_1469996503.log


Finished parsing file /home/ubuntu/TalkingData/source_data/app_labels.csv

Parsing completed. Parsed 100 lines in 0.090849 secs.

...

Finished parsing file /home/ubuntu/TalkingData/source_data/events.csv

Parsing completed. Parsed 100 lines in 0.29242 secs.

Read 884193 lines. Lines per second: 1.39602e+06

Finished parsing file /home/ubuntu/TalkingData/source_data/events.csv

Parsing completed. Parsed 3252950 lines in 1.65391 secs.

In [2]:
gender_age_train.head(3)

device_id,gender,age,group
-8076087639492063270,M,35,M32-38
-2897161552818060146,M,35,M32-38
-8260683887967679142,M,35,M32-38


## Creating training and testing set

In [3]:
ga_train, ga_test = gender_age_train.random_split(0.8, seed=5)

## Benchmark score

In [4]:
benchmark = ga_train.groupby('group',{'count':gl.aggregate.COUNT('device_id')})
benchmark

group,count
F33-42,4449
F23-,4083
F27-28,2515
M27-28,4424
M32-38,7575
F29-32,3711
F24-26,3367
M23-26,7692
M29-31,5820
M39+,6877


In [5]:
total = benchmark['count'].sum()
benchmark['freq']=benchmark['count']/total
classes = sorted(sample_submission.column_names())
classes.remove("device_id")
submission = gl.SFrame({'device_id':ga_test['device_id']})

In [6]:
for i, label in enumerate(classes):
    submission[label] = benchmark[benchmark['group']==label]['freq'][0]
submission.head(1)

Columns:
	device_id	str
	F23-	float
	F24-26	float
	F27-28	float
	F29-32	float
	F33-42	float
	F43+	float
	M22-	float
	M23-26	float
	M27-28	float
	M29-31	float
	M32-38	float
	M39+	float

Rows: 1

Data:
+--------------------+-----------------+-----------------+-----------------+
|     device_id      |       F23-      |      F24-26     |      F27-28     |
+--------------------+-----------------+-----------------+-----------------+
| 236877999787307864 | 0.0681954836986 | 0.0562366381614 | 0.0420062800641 |
+--------------------+-----------------+-----------------+-----------------+
+-----------------+----------------+-----------------+-----------------+
|      F29-32     |     F33-42     |       F43+      |       M22-      |
+-----------------+----------------+-----------------+-----------------+
| 0.0619822287547 | 0.074308524853 | 0.0566207910208 | 0.0996960181721 |
+-----------------+----------------+-----------------+-----------------+
+----------------+-----------------+-----------------+----------------+
|     M23-26     |      M27-28     |      M29-31     |     M32-38     |
+----------------+-----------------+-----------------+----------------+
| 0.128474078033 | 0.0738909673971 | 0.0972073757349 | 0.126519909139 |
+----------------+-----------------+-----------------+----------------+
+----------------+
|      M39+      |
+----------------+
| 0.114861704971 |
+----------------+
[1 rows x 13 columns]

In [7]:
target = ga_test['group']
predictions = submission.remove_column('device_id')
row = []
col = []
for label in classes:
    row.append(predictions[0][label])
for i in range(len(ga_test)):
    col.append(row)
submission = gl.SArray(col)

In [8]:
log_loss = gl.evaluation.log_loss(target, submission)
print 'the benchmark logloss is {}'.format(log_loss)

the benchmark logloss is 2.4234927185


## Merging data into one table

In [9]:
app_label_list = app_labels.groupby('app_id',{'label_list': gl.aggregate.CONCAT('label_id')})
app_label_list.head(3)

app_id,label_list
-539730114641165062,"[548, 710, 704]"
-736899763926984371,"[405, 796, 794, 795]"
3258653066506014655,"[548, 549, 128]"


In [10]:
app_events = app_events.join(app_label_list, on='app_id', how='inner')
app_events['label_list'] = app_events['label_list'].apply(lambda x: ' '.join(str(e) for e in x))

In [11]:
# separate app_events
app_events_installed = app_events[app_events['is_active']==0]
app_events_active = app_events[app_events['is_active']==1]

In [12]:
ga_events_active = app_events_active.groupby("event_id", {"active_label": gl.aggregate.CONCAT('label_list'), 'active_app_id': gl.aggregate.CONCAT('app_id')})
ga_events_active['active_label'] =ga_events_active['active_label'].apply(lambda x: ' '.join(str(e) for e in x))
ga_events_active['active_tfidf']= gl.text_analytics.tf_idf(ga_events_active['active_label'])
ga_events_active.head(3)

event_id,active_label,active_app_id,active_tfidf
2658191,549 710 169 271 302 548303 318 704 306 843 840 ...,"[5195654633279707117,-6793861127573349654] ...","{'318':3.0342132763787455, ..."
846413,548 549 152 548 704 713717 721 232 721 548 704 ...,"[543880124725657021,-755461362045697404, ...","{'152': 2.67227674683976,'704': 1.184211267210 ..."
2683136,548 549 549 710 548 704172 548 549 714 548 704 ...,"[-7680145830980282919,5927333115845830913, ...","{'704':0.7401320420064275, ..."


In [13]:
ga_events_install = app_events_installed.groupby("event_id", {"installed_label": gl.aggregate.CONCAT('label_list'), 'installed_app_id': gl.aggregate.CONCAT('app_id')})
ga_events_install['installed_label'] =ga_events_install['installed_label'].apply(lambda x: ' '.join(str(e) for e in x))
ga_events_install['install_tfidf']= gl.text_analytics.tf_idf(ga_events_install['installed_label'])
ga_events_install.head(3)

event_id,installed_label,installed_app_id,install_tfidf
561271,549 405 548 730 756 761777 782 787 1012 713 704 ...,"[3722410295315095646,133935734026685494, ...","{'704':0.04857517511347787, ..."
1718105,549 710 548 704 172 548549 168 549 721 724 186 ...,"[5927333115845830913,-1805603084550565936, ...","{'214':1.6798201754695075, ..."
895926,549 713 237 704 548 549721 302 303 704 548 183 ...,"[8483751493632839871,-7377004479023402858, ...","{'775':0.9557122571870953, ..."


In [14]:
events = events.join(ga_events_active, on ='event_id', how='left') # events has more event id than app_event, so use left join
events= events.join(ga_events_install, on = 'event_id', how='left')
events.head(3)

Columns:
	event_id	int
	device_id	str
	timestamp	str
	longitude	float
	latitude	float
	active_label	str
	active_app_id	list
	active_tfidf	dict
	installed_label	str
	installed_app_id	list
	install_tfidf	dict

Rows: 3

Data:
+----------+----------------------+---------------------+-----------+----------+
| event_id |      device_id       |      timestamp      | longitude | latitude |
+----------+----------------------+---------------------+-----------+----------+
|    1     |  29182687948017175   | 2016-05-01 00:55:25 |   121.38  |  31.24   |
|    2     | -6401643145415154744 | 2016-05-01 00:54:12 |   103.65  |  30.97   |
|    3     | -4833982096941402721 | 2016-05-01 00:08:05 |   106.6   |   29.7   |
+----------+----------------------+---------------------+-----------+----------+
+-------------------------------+-------------------------------+
|          active_label         |         active_app_id         |
+-------------------------------+-------------------------------+
|              None             |              None             |
| 549 710 548 704 172 405 73... | [5927333115845830913, -653... |
|              None             |              None             |
+-------------------------------+-------------------------------+
+-------------------------------+-------------------------------+
|          active_tfidf         |        installed_label        |
+-------------------------------+-------------------------------+
|              None             |              None             |
| {'747': 4.9236469360864525... | 721 548 302 303 704 251 26... |
|              None             |              None             |
+-------------------------------+-------------------------------+
+-------------------------------+-------------------------------+
|        installed_app_id       |         install_tfidf         |
+-------------------------------+-------------------------------+
|              None             |              None             |
| [-5720078949152207372, -16... | {'704': 0.0850065564485862... |
|              None             |              None             |
+-------------------------------+-------------------------------+
[3 rows x 11 columns]

In [15]:
def period (x):
    if x.hour>=0 and x.hour<=5:
        return 'midnight'
    elif x.hour>=6 and x.hour<=11:
        return 'morning'
    elif x.hour>=12 and x.hour<=17:
        return 'afternoon'
    else:
        return 'evening'

In [16]:
name_of_weekday = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturaday','Sunday']
def weekday (x):
    return name_of_weekday[x-1]

In [17]:
# creating time features
events["timestamp"] = events["timestamp"].str_to_datetime(str_format="%Y-%m-%dT%H:%M:%S")
events['weekday']= events['timestamp'].apply(lambda x: x.isoweekday())
events['weekday']= events['weekday'].apply(weekday)
events['period'] = events['timestamp'].apply(period)

In [18]:
# adding phone brand device model 
events = events.join(phone_brand_device_model, on='device_id',how='inner')

In [19]:
events.head(3)

Columns:
	event_id	int
	device_id	str
	timestamp	datetime
	longitude	float
	latitude	float
	active_label	str
	active_app_id	list
	active_tfidf	dict
	installed_label	str
	installed_app_id	list
	install_tfidf	dict
	weekday	str
	period	str
	phone_brand	str
	device_model	str

Rows: 3

Data:
+----------+----------------------+---------------------+-----------+----------+
| event_id |      device_id       |      timestamp      | longitude | latitude |
+----------+----------------------+---------------------+-----------+----------+
|    1     |  29182687948017175   | 2016-05-01 00:55:25 |   121.38  |  31.24   |
|    2     | -6401643145415154744 | 2016-05-01 00:54:12 |   103.65  |  30.97   |
|    3     | -4833982096941402721 | 2016-05-01 00:08:05 |   106.6   |   29.7   |
+----------+----------------------+---------------------+-----------+----------+
+-------------------------------+-------------------------------+
|          active_label         |         active_app_id         |
+-------------------------------+-------------------------------+
|              None             |              None             |
| 549 710 548 704 172 405 73... | [5927333115845830913, -653... |
|              None             |              None             |
+-------------------------------+-------------------------------+
+-------------------------------+-------------------------------+
|          active_tfidf         |        installed_label        |
+-------------------------------+-------------------------------+
|              None             |              None             |
| {'747': 4.9236469360864525... | 721 548 302 303 704 251 26... |
|              None             |              None             |
+-------------------------------+-------------------------------+
+-------------------------------+-------------------------------+---------+
|        installed_app_id       |         install_tfidf         | weekday |
+-------------------------------+-------------------------------+---------+
|              None             |              None             |  Sunday |
| [-5720078949152207372, -16... | {'704': 0.0850065564485862... |  Sunday |
|              None             |              None             |  Sunday |
+-------------------------------+-------------------------------+---------+
+----------+-------------+--------------------+
|  period  | phone_brand |    device_model    |
+----------+-------------+--------------------+
| midnight |     小米    |      红米note      |
| midnight |     三星    | Galaxy Grand Prime |
| midnight |     魅族    |      MX4 Pro       |
+----------+-------------+--------------------+
[3 rows x 15 columns]

In [20]:
# merging events into training and testing dataset
ga_train = ga_train.join(events, on='device_id', how='inner')
ga_train.head(3)

Columns:
	device_id	str
	gender	str
	age	int
	group	str
	event_id	int
	timestamp	datetime
	longitude	float
	latitude	float
	active_label	str
	active_app_id	list
	active_tfidf	dict
	installed_label	str
	installed_app_id	list
	install_tfidf	dict
	weekday	str
	period	str
	phone_brand	str
	device_model	str

Rows: 3

Data:
+----------------------+--------+-----+-------+----------+---------------------+
|      device_id       | gender | age | group | event_id |      timestamp      |
+----------------------+--------+-----+-------+----------+---------------------+
| -8195816569128397698 |   M    |  39 |  M39+ |    10    | 2016-05-01 00:41:31 |
| 6130108008013735751  |   F    |  45 |  F43+ |    20    | 2016-05-01 00:17:33 |
| -1663840927569383079 |   F    |  19 |  F23- |    22    | 2016-05-01 00:43:15 |
+----------------------+--------+-----+-------+----------+---------------------+
+-----------+----------+--------------+---------------+--------------+-----------------+
| longitude | latitude | active_label | active_app_id | active_tfidf | installed_label |
+-----------+----------+--------------+---------------+--------------+-----------------+
|   119.34  |  26.04   |     None     |      None     |     None     |       None      |
|   129.51  |  42.91   |     None     |      None     |     None     |       None      |
|   116.45  |  33.93   |     None     |      None     |     None     |       None      |
+-----------+----------+--------------+---------------+--------------+-----------------+
+------------------+---------------+---------+----------+-------------+
| installed_app_id | install_tfidf | weekday |  period  | phone_brand |
+------------------+---------------+---------+----------+-------------+
|       None       |      None     |  Sunday | midnight |     锤子    |
|       None       |      None     |  Sunday | midnight |     酷派    |
|       None       |      None     |  Sunday | midnight |     vivo    |
+------------------+---------------+---------+----------+-------------+
+--------------+
| device_model |
+--------------+
|   坚果手机   |
|    大神F1    |
|    X5Max+    |
+--------------+
[3 rows x 18 columns]

In [21]:
ga_test = ga_test.join(events, on='device_id', how='inner')

## Applying Gradient boosting decision trees

In [22]:
# Creating validation set
import time
ga_train_id = gl.SFrame({'device_id':ga_train['device_id'].unique()})
ga_train_id = gl.cross_validation.shuffle(ga_train_id, random_seed=5)
folds = gl.cross_validation.KFold(ga_train_id, num_folds=5)

In [23]:
fold_train, fold_val = folds[0]
fold_train = fold_train.join(ga_train, on='device_id')
fold_val = fold_val.join(ga_train, on='device_id')

In [24]:
# make sure no device_id overlapping between train and validation set
assert set(fold_val['device_id'].unique()) & set(fold_train['device_id'].unique()) == set()

### 1) app features

In [25]:
# feature selection trials
target = "group"
features = ['active_tfidf']
model = gl.classifier.boosted_trees_classifier.create(fold_train,
                                                      target=target, features=features,
                                                      validation_set=fold_val, max_iterations = 50)

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 770048

Number of classes           : 12

Number of feature columns   : 1

...

| 47        | 172.168482   | 0.254575          | 2.130550          | 0.195980            | 2.277107            |

| 48        | 175.921098   | 0.255030          | 2.129317          | 0.196056            | 2.277237            |

| 49        | 179.729829   | 0.255897          | 2.127505          | 0.196097            | 2.277062            |

| 50        | 183.489921   | 0.256414          | 2.125945          | 0.195229            | 2.277437            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

In [66]:
target = "group"
features = ['install_tfidf']
model = gl.classifier.boosted_trees_classifier.create(fold_train,
                                                      target=target, features=features,
                                                      validation_set=fold_val, max_iterations = 50)

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 770373

Number of classes           : 12

Number of feature columns   : 1

...

| 47        | 208.391876   | 0.292426          | 2.069352          | 0.188606            | 2.289780            |

| 48        | 212.810243   | 0.292826          | 2.068044          | 0.188342            | 2.290380            |

| 49        | 217.102493   | 0.293351          | 2.066295          | 0.188433            | 2.290678            |

| 50        | 221.634582   | 0.293806          | 2.065000          | 0.188784            | 2.290450            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

In [67]:
target = "group"
features = ['active_label']
model = gl.classifier.boosted_trees_classifier.create(fold_train,
                                                      target=target, features=features,
                                                      validation_set=fold_val, max_iterations = 50)

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 770373

Number of classes           : 12

Number of feature columns   : 1

...

| 47        | 65.847078    | 0.207031          | 2.282114          | 0.181319            | 2.313333            |

| 48        | 67.220141    | 0.207168          | 2.281687          | 0.181319            | 2.313331            |

| 49        | 68.594352    | 0.207379          | 2.281265          | 0.181299            | 2.313299            |

| 50        | 70.015979    | 0.207583          | 2.280849          | 0.181365            | 2.313308            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

In [89]:
target = "group"
features = ['installed_label']
model = gl.classifier.boosted_trees_classifier.create(fold_train,
                                                      target=target, features=features,
                                                      validation_set=fold_val, max_iterations = 50)

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 770373

Number of classes           : 12

Number of feature columns   : 1

...

| 47        | 53.310247    | 0.200905          | 2.296918          | 0.174256            | 2.320592            |

| 48        | 54.441217    | 0.201186          | 2.296602          | 0.174256            | 2.320587            |

| 49        | 55.565259    | 0.201384          | 2.296289          | 0.174256            | 2.320584            |

| 50        | 56.686660    | 0.201532          | 2.295983          | 0.174256            | 2.320582            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

In [69]:
target = "group"
features = ['active_app_id']
model = gl.classifier.boosted_trees_classifier.create(fold_train,
                                                      target=target, features=features,
                                                      validation_set=fold_val, max_iterations = 50)

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 770373

Number of classes           : 12

Number of feature columns   : 1

...

| 47        | 62.540631    | 0.344397          | 1.976549          | 0.195349            | 2.279294            |

| 48        | 63.868289    | 0.345411          | 1.974454          | 0.195252            | 2.279433            |

| 49        | 65.184107    | 0.345825          | 1.972458          | 0.195425            | 2.279856            |

| 50        | 66.494215    | 0.346579          | 1.970557          | 0.195450            | 2.279656            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

In [70]:
target = "group"
features = ['installed_app_id']
model = gl.classifier.boosted_trees_classifier.create(fold_train,
                                                      target=target, features=features,
                                                      validation_set=fold_val, max_iterations = 50)

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 770373

Number of classes           : 12

Number of feature columns   : 1

...

| 47        | 87.087759    | 0.287084          | 2.102897          | 0.190669            | 2.281996            |

| 48        | 88.927141    | 0.287384          | 2.101635          | 0.190730            | 2.281993            |

| 49        | 90.759445    | 0.287832          | 2.100422          | 0.190633            | 2.281910            |

| 50        | 92.601308    | 0.288276          | 2.099236          | 0.190659            | 2.281642            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

### 2) time feature

In [71]:
target = "group"
features = ['period']
model = gl.classifier.boosted_trees_classifier.create(fold_train,
                                                      target=target, features=features,
                                                      validation_set=fold_val, max_iterations = 50)

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 770373

Number of classes           : 12

Number of feature columns   : 1

...

| 35        | 22.907228    | 0.174913          | 2.338770          | 0.177295            | 2.319000            |

| 40        | 25.892750    | 0.174913          | 2.338769          | 0.177295            | 2.318974            |

| 45        | 28.749575    | 0.174913          | 2.338769          | 0.177295            | 2.318963            |

| 50        | 31.443911    | 0.174913          | 2.338769          | 0.177295            | 2.318958            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

In [72]:
target = "group"
features = ['weekday']
model = gl.classifier.boosted_trees_classifier.create(fold_train,
                                                      target=target, features=features,
                                                      validation_set=fold_val, max_iterations = 50)

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 770373

Number of classes           : 12

Number of feature columns   : 1

...

| 35        | 26.466195    | 0.174913          | 2.339279          | 0.177295            | 2.320786            |

| 40        | 29.788640    | 0.174913          | 2.339278          | 0.177295            | 2.320760            |

| 45        | 32.889253    | 0.174913          | 2.339278          | 0.177295            | 2.320749            |

| 50        | 35.691710    | 0.174913          | 2.339278          | 0.177295            | 2.320744            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

In [73]:
target = "group"
features = ['period','weekday']
model = gl.classifier.boosted_trees_classifier.create(fold_train,
                                                      target=target, features=features,
                                                      validation_set=fold_val, max_iterations = 50)

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 770373

Number of classes           : 12

Number of feature columns   : 2

...

| 35        | 30.214191    | 0.177010          | 2.335329          | 0.177300            | 2.322306            |

| 40        | 34.499065    | 0.177010          | 2.335326          | 0.177300            | 2.322306            |

| 45        | 38.495989    | 0.177010          | 2.335326          | 0.177300            | 2.322311            |

| 50        | 42.068508    | 0.177010          | 2.335326          | 0.177300            | 2.322312            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

### 3) location features

In [75]:
target = "group"
features = ['longitude']
model = gl.classifier.boosted_trees_classifier.create(fold_train,
                                                      target=target, features=features,
                                                      validation_set=fold_val, max_iterations = 50)

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 770373

Number of classes           : 12

Number of feature columns   : 1

...

| 47        | 64.816808    | 0.415492          | 1.796637          | 0.143906            | 2.466549            |

| 48        | 66.194067    | 0.416240          | 1.792842          | 0.143937            | 2.469168            |

| 49        | 67.566964    | 0.416901          | 1.790472          | 0.143825            | 2.472147            |

| 50        | 68.927214    | 0.417844          | 1.787450          | 0.143047            | 2.473943            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

In [76]:
target = "group"
features = ['latitude']
model = gl.classifier.boosted_trees_classifier.create(fold_train,
                                                      target=target, features=features,
                                                      validation_set=fold_val, max_iterations = 50)

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 770373

Number of classes           : 12

Number of feature columns   : 1

...

| 47        | 56.586174    | 0.415309          | 1.798550          | 0.145344            | 2.448245            |

| 48        | 57.805413    | 0.416011          | 1.795608          | 0.145466            | 2.450140            |

| 49        | 58.989546    | 0.416018          | 1.792928          | 0.145959            | 2.451716            |

| 50        | 60.168999    | 0.416250          | 1.790112          | 0.146101            | 2.455246            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

In [77]:
target = "group"
features = ['longitude','latitude']
model = gl.classifier.boosted_trees_classifier.create(fold_train,
                                                      target=target, features=features,
                                                      validation_set=fold_val, max_iterations = 50)

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 770373

Number of classes           : 12

Number of feature columns   : 2

...

| 47        | 62.529917    | 0.562818          | 1.470344          | 0.160898            | 2.481246            |

| 48        | 63.817774    | 0.564576          | 1.464630          | 0.160771            | 2.482091            |

| 49        | 65.151340    | 0.566293          | 1.459318          | 0.161370            | 2.485055            |

| 50        | 66.453682    | 0.568284          | 1.454253          | 0.161264            | 2.486326            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

### 4) phone brand device model features

In [79]:
target = "group"
features = ['phone_brand']
model = gl.classifier.boosted_trees_classifier.create(fold_train,
                                                      target=target, features=features,
                                                      validation_set=fold_val, max_iterations = 50)

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 770373

Number of classes           : 12

Number of feature columns   : 1

...

| 35        | 29.080689    | 0.199901          | 2.269879          | 0.190704            | 2.301966            |

| 40        | 33.145989    | 0.199910          | 2.269207          | 0.190831            | 2.302693            |

| 45        | 37.198411    | 0.199910          | 2.268694          | 0.190831            | 2.303591            |

| 50        | 41.244873    | 0.199912          | 2.268308          | 0.190831            | 2.304245            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

In [80]:
target = "group"
features = ['device_model']
model = gl.classifier.boosted_trees_classifier.create(fold_train,
                                                      target=target, features=features,
                                                      validation_set=fold_val, max_iterations = 50)

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 770373

Number of classes           : 12

Number of feature columns   : 1

...

| 35        | 28.182077    | 0.306921          | 2.073324          | 0.175369            | 2.348279            |

| 40        | 32.165162    | 0.309398          | 2.063208          | 0.172574            | 2.352152            |

| 45        | 36.135189    | 0.311600          | 2.054273          | 0.178159            | 2.355129            |

| 50        | 40.108831    | 0.313320          | 2.046357          | 0.176944            | 2.358934            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

In [81]:
target = "group"
features = ['phone_brand','device_model']
model = gl.classifier.boosted_trees_classifier.create(fold_train,
                                                      target=target, features=features,
                                                      validation_set=fold_val, max_iterations = 50)

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 770373

Number of classes           : 12

Number of feature columns   : 2

...

| 35        | 33.513334    | 0.311132          | 2.053259          | 0.162138            | 2.344954            |

| 40        | 38.154659    | 0.313222          | 2.044196          | 0.166015            | 2.349335            |

| 45        | 43.210114    | 0.313669          | 2.035986          | 0.165029            | 2.354456            |

| 50        | 47.836710    | 0.315230          | 2.028670          | 0.166004            | 2.360280            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

## final model

In [41]:
target = "group"
features = ['longitude', 'latitude', 'active_tfidf', 'install_tfidf','phone_brand','device_model','weekday','period']
model2 = gl.classifier.boosted_trees_classifier.create(fold_train,
                                                      target=target, features=features,
                                                      validation_set=fold_val, max_iterations = 100, max_depth=6,
                                                      step_size=0.1, column_subsample = 0.3)

Boosted trees classifier:

--------------------------------------------------------

Number of examples          : 770048

Number of classes           : 12

Number of feature columns   : 8

...

| 97        | 373.724081   | 0.565297          | 1.587322          | 0.203047            | 2.274623            |

| 98        | 377.708568   | 0.566449          | 1.584362          | 0.203285            | 2.275015            |

| 99        | 381.487425   | 0.567653          | 1.581590          | 0.203067            | 2.275071            |

| 100       | 385.431703   | 0.568533          | 1.578709          | 0.202732            | 2.275107            |

+-----------+--------------+-------------------+-------------------+---------------------+---------------------+

## cross validation

In [47]:
log_loss_values = []
for i in range(5):
    fold_train, fold_val = folds[i]
    fold_train = fold_train.join(ga_train, on='device_id')
    fold_val = fold_val.join(ga_train, on='device_id')
    target = "group"
    features = ['longitude', 'latitude', 'active_tfidf', 'install_tfidf','phone_brand','device_model','weekday','period']
    model2 = gl.classifier.boosted_trees_classifier.create(fold_train,
                                                      target=target, features=features,
                                                      validation_set=fold_val, max_iterations = 100, max_depth=6,
                                                      step_size=0.1, column_subsample = 0.3,verbose=False)
    
    log_loss_values.append(model2.evaluate(fold_val, metric="log_loss")["log_loss"])

for i, ll in enumerate(log_loss_values):
    print "log loss for fold %d: %f" %(i+1, ll)

log loss for fold 1: 2.275107
log loss for fold 2: 2.255033
log loss for fold 3: 2.271240
log loss for fold 4: 2.266012
log loss for fold 5: 2.250840


## score on test set

In [42]:
model2.evaluate(ga_test, metric="log_loss")

{'log_loss': 2.254168160891516}